In [4]:
from dataloader import dataloader, SignalDataset
from utils import prepare_data
from param import dataset_path, sample_universe_size
from torch.utils.data import ConcatDataset
from MTL_w_cascade_info import MtlCascadeModel
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR
import torch

# from torch.nn.utils import weight_norm
from torch import nn


hp = {
    "n_layers": 1,
    "sp_hidden_nodes": 20,
    "n_sp_hidden_lyrs": 1,
    "mu_hidden_nodes": 20,
    "n_mu_hidden_lyrs": 1,
    "smr_hidden_nodes": 20,
    "n_smr_hidden_lyrs": 1,
    "n_epochs": 100,
    "batch_size": 20,
    "train_ratio": 0.8,
}
combined_dataset = torch.load('dataset.pth')
print("data_loader")
train_loader, test_loader = dataloader(
    datasets=combined_dataset,
    train_ratio=hp["train_ratio"],
    train_batch_size=hp["batch_size"],
    test_batch_size=1,
)

def train(
    train_loader,
    model,
    epoch,
    out_dict,
    loss_sp_fn,
    loss_mu_fn,
    loss_smr_fn,
    optimizer,
):
    correct = 0
    for data in train_loader:
        feature, label = data
        y = [out_dict[x] for x in label]
        out_sp, out_mu, out_smr = model(feature)

        sp_list = [inner_list[0] for inner_list in y]
        mu_list = [inner_list[1] for inner_list in y]
        smr_list = [inner_list[2:] for inner_list in y]
        y_sp = torch.Tensor(sp_list).unsqueeze(1)
        y_mu = torch.Tensor(mu_list).unsqueeze(1)
        y_smr = torch.Tensor(smr_list).unsqueeze(1)

        loss_sp = loss_sp_fn(out_sp, y_sp)
        loss_mu = loss_mu_fn(out_mu, y_mu)
        loss_smr = loss_smr_fn(out_smr, y_smr)

        total_loss = loss_sp + loss_mu + loss_smr
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        pred_y = torch.cat((out_sp, out_mu, out_smr), dim=1)
        result = (pred_y > 0.5).float()
        target = torch.tensor(y).float()
        for i in range(result.size(0)):
            if torch.all(torch.eq(result[i], target[i])):
                correct += 1
    accuracy = correct / len(train_loader.dataset)
    print(
        f"Epoch: {epoch}, Loss_sp: {loss_sp}, Loss_mu: {loss_mu}, Loss_smr: {loss_smr}, Accuracy: {accuracy}"
    )


data_loader


In [6]:
torch.autograd.set_detect_anomaly(True)

print("model_init")
model = MtlCascadeModel(hp)
# print(model)
loss_sp = nn.BCEWithLogitsLoss()
loss_mu = nn.BCEWithLogitsLoss()
loss_smr = nn.MSELoss()

out_dict = {"speech": [1, 0, 0, 0], "music": [0, 1, 0, 0], "mixture": [0, 0, 1, 1]}

# Optimizer and learning rate scheduler
optimizer = Adam(model.parameters(), lr=0.002)
# optimizer = SGD(model.parameters(), lr=0.002, momentum=0.9)
scheduler = ExponentialLR(optimizer, gamma=0.1)

print("start_training")
for epoch in range(1,hp["n_epochs"]+1):
    train(
        train_loader, model, epoch, out_dict, loss_sp, loss_mu, loss_smr, optimizer
    )
# train(
#     train_loader, model, hp["n_epochs"], out_dict, loss_sp, loss_mu, loss_smr, optimizer
# )

model_init
start_training


/home/prateeks/projects/iitj/su/Speech-Understanding-Minor/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([20, 1, 2])) that is different to the input size (torch.Size([20, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/prateeks/projects/iitj/su/Speech-Understanding-Minor/.venv/lib/python3.9/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8, 1, 2])) that is different to the input size (torch.Size([8, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1, Loss_sp: 0.5497570037841797, Loss_mu: 1.669691562652588, Loss_smr: 0.08134792745113373, Accuracy: 0.22821576763485477
Epoch: 2, Loss_sp: 0.12731775641441345, Loss_mu: 0.3549463748931885, Loss_smr: 0.1975173056125641, Accuracy: 0.36751630112625966
Epoch: 3, Loss_sp: 0.13728009164333344, Loss_mu: 0.4615171551704407, Loss_smr: 0.19479364156723022, Accuracy: 0.4013040901007706
Epoch: 4, Loss_sp: 0.18401402235031128, Loss_mu: 0.39979904890060425, Loss_smr: 0.3128724694252014, Accuracy: 0.41567871962062836
Epoch: 5, Loss_sp: 0.07096236944198608, Loss_mu: 0.4647606313228607, Loss_smr: 0.23608353734016418, Accuracy: 0.40915826911677533
Epoch: 6, Loss_sp: 0.35188257694244385, Loss_mu: 0.545046865940094, Loss_smr: 0.19359494745731354, Accuracy: 0.43375815056312983
Epoch: 7, Loss_sp: 0.19236329197883606, Loss_mu: 0.5008758902549744, Loss_smr: 0.2694648802280426, Accuracy: 0.4350918790752816
Epoch: 8, Loss_sp: 1.4852579832077026, Loss_mu: 0.21463799476623535, Loss_smr: 0.1586016565561294